# Deploy Web App on Azure Container Services (AKS)


In this notebook, we will set up an Azure Container Service which will be managed by Kubernetes. We will then take the Docker image we created earlier that contains our app and deploy it to the AKS cluster. Then, we will check everything is working by sending an image to it and getting it scored. 

The process is split into the following steps:
- Define our resource names
- Login to Azure
- Create resource group and create AKS
- Connect to AKS
- Deploy our app
- Tear it all down

We assume that this notebook is running on Linux and Azure CLI is installed before proceeding.

## Setup

Below are the various name definitions for the resources needed to setup AKS as well as the name of the Docker image we will be using.

In [1]:
# Please modify the below as you see fit
resource_group = "<RESOURCE_GROUP>" 
aks_name = "<AKS_CLUSTER_NAME>"
location = "eastus"

image_name = '<YOUR_DOCKER_IMAGE>' # 'fboylu/kerastf-gpu' Feel free to use this image if you want to 
                                   # skip creating your own container
selected_subscription = "'<YOUR_SUBSCRIPTION>'" # If you have multiple subscriptions select 
                                                # the subscription you want to use here

In [1]:
resource_group = "fbakscpurg" # Feel free to modify these
aks_name = "fbAKSClustercpu"
location = "eastus"

image_name = "fboylu/kerasres50-cpu" 
selected_subscription = "'Team Danielle Internal'"

## Azure account login

The command below will initiate a login to your Azure account. It will pop up with an url to go to where you will enter a one off code and log into your Azure account using your browser.

In [6]:
!az login -o table

In [7]:
!az account set --subscription $selected_subscription

In [2]:
!az account show

{
  "environmentName": "AzureCloud",
  "id": "edf507a2-6235-46c5-b560-fd463ba2e771",
  "isDefault": true,
  "name": "Team Danielle Internal",
  "state": "Enabled",
  "tenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
  "user": {
    "name": "fboylu@microsoft.com",
    "type": "user"
  }
}


You will also need to register the container service resources on your subscription if you haven't already done so.

In [9]:
!az provider register -n Microsoft.ContainerService

In [10]:
!az provider show -n Microsoft.ContainerService

## Create resources and dependencies

### Create resource group and AKS cluster

Azure encourages the use of groups to organize all the Azure components you deploy. That way it is easier to find them but also we can delete a number of resources simply by deleting the group.

In [3]:
!az group create --name $resource_group --location $location

{
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourceGroups/fbaksrg",
  "location": "eastus",
  "managedBy": null,
  "name": "fbaksrg",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null
}


Below, we create the AKS cluster in the resource group we created earlier.

In [6]:
%%time
!az aks create --resource-group $resource_group --name $aks_name --node-count 1 --generate-ssh-keys -s Standard_D4_v2

{- Finished ..
  "agentPoolProfiles": [
    {
      "count": 1,
      "dnsPrefix": null,
      "fqdn": null,
      "name": "nodepool1",
      "osDiskSizeGb": null,
      "osType": "Linux",
      "ports": null,
      "storageProfile": "ManagedDisks",
      "vmSize": "Standard_NC6",
      "vnetSubnetId": null
    }
  ],
  "dnsPrefix": "fbAKSClust-fbaksrg-edf507",
  "fqdn": "fbaksclust-fbaksrg-edf507-e12d7f40.hcp.eastus.azmk8s.io",
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourcegroups/fbaksrg/providers/Microsoft.ContainerService/managedClusters/fbAKSClustergpu",
  "kubernetesVersion": "1.9.6",
  "linuxProfile": {
    "adminUsername": "azureuser",
    "ssh": {
      "publicKeys": [
        {
          "keyData": "ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQDTZYQFHNstYCR25qtvMrC6baTMS6TobaIRbgd0xOoafDy+2uBk0DMJuhGWoOcrsCnvadp5k/0K8qBRysyhlQGWb6+r8fBunThy+zpTKqdh3W8Q1y5UtKnGwwU1cqGXDOPUIXJYNPJqUKV829+MOrZjUynhHgSzDbY2ncGyoT+Farsvm01aGEdDapa+XRl4JAwtN1bb9q+Ii5y+MkpIOhLRMwATl05

### Install kubectl CLI

To connect to the Kubernetes cluster, we will use kubectl, the Kubernetes command-line client. To install, run the following:

In [3]:
!sudo az aks install-cli

Please ensure that /usr/local/bin is in your search PATH, so the `kubectl` command can be found.


## Connect to AKS cluster

To configure kubectl to connect to the Kubernetes cluster, run the following command:

In [4]:
!az aks get-credentials --resource-group $resource_group --name $aks_name

Merged "fbAKSClustercpu" as current context in /home/fboylu/.kube/config


Let's verify connection by listing the nodes.

In [5]:
!kubectl get nodes

NAME                       STATUS    ROLES     AGE       VERSION
aks-nodepool1-13347348-0   Ready     agent     6d        v1.9.6
aks-nodepool1-13347348-1   Ready     agent     6d        v1.9.6
aks-nodepool1-13347348-2   Ready     agent     6d        v1.9.6
aks-nodepool1-13347348-3   Ready     agent     6d        v1.9.6
aks-nodepool1-13347348-4   Ready     agent     6d        v1.9.6


Let's check the pods on our cluster.

In [13]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
kube-system   azureproxy-79c5db744-j5hj4              1/1       Running   3          6d
kube-system   heapster-55f855b47-bs86x                2/2       Running   0          6d
kube-system   kube-dns-v20-7c556f89c5-96dfd           3/3       Running   0          6d
kube-system   kube-dns-v20-7c556f89c5-fm7v2           3/3       Running   0          6d
kube-system   kube-proxy-225hz                        1/1       Running   0          6d
kube-system   kube-proxy-g5x5p                        1/1       Running   0          6d
kube-system   kube-proxy-hw7fk                        1/1       Running   0          6d
kube-system   kube-proxy-ttnsh                        1/1       Running   0          6d
kube-system   kube-proxy-tvnpv                        1/1       Running   0          6d
kube-system   kube-svc-redirect-278jk                 1/1       Running   0          6d
kube-system   kube-svc-redirect

## Deploy application

Below we define our Kubernetes manifest file for our service and load balancer. Note that we have to specify the volume mounts to the drivers that are located on the node.

In [14]:
app_template = {
  "apiVersion": "apps/v1beta1",
  "kind": "Deployment",
  "metadata": {
      "name": "azure-dl"
  },
  "spec":{
      "replicas":2,
      "template":{
          "metadata":{
              "labels":{
                  "app":"azure-dl"
              }
          },
          "spec":{
              "containers":[
                  {
                      "name": "azure-dl",
                      "image": "fboylu/kerasres50-cpu",

                      "ports":[
                          {
                              "containerPort":80,
                              "name":"model"
                          }
                      ],
                      "resources":{
                           "requests":{
                               "cpu": 1
                           },
                           "limits":{
                               "cpu": 1.25
                           }
                       }  
                  }
              ]
          }
      }
  }
}

service_temp = {
  "apiVersion": "v1",
  "kind": "Service",
  "metadata": {
      "name": "azure-dl"
  },
  "spec":{
      "type": "LoadBalancer",
      "ports":[
          {
              "port":80
          }
      ],
      "selector":{
            "app":"azure-dl"
      }
   }
}

In [15]:
import json
def write_json_to_file(json_dict, filename, mode='w'):
    with open(filename, mode) as outfile:
        json.dump(json_dict, outfile, indent=4, sort_keys=True)
        outfile.write('\n\n')

In [16]:
write_json_to_file(app_template, 'az-dl.json')

In [17]:
write_json_to_file(service_temp, 'az-dl.json', mode='a')

Let's check the manifest created.

In [18]:
!cat az-dl.json

{
    "apiVersion": "apps/v1beta1",
    "kind": "Deployment",
    "metadata": {
        "name": "azure-dl"
    },
    "spec": {
        "replicas": 2,
        "template": {
            "metadata": {
                "labels": {
                    "app": "azure-dl"
                }
            },
            "spec": {
                "containers": [
                    {
                        "image": "fboylu/kerasres50-cpu",
                        "name": "azure-dl",
                        "ports": [
                            {
                                "containerPort": 80,
                                "name": "model"
                            }
                        ],
                        "resources": {
                            "limits": {
                                "cpu": 1.25
                            },
                            "requests": {
                                "cpu": 1
                            }
                        }
        

Next, we will use kubectl create command to deploy our application.

In [19]:
!kubectl create -f az-dl.json

deployment.apps "azure-dl" created
service "azure-dl" created


Let's check if the pod is deployed.

In [28]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
default       azure-dl-6bcd9d5bfb-2txj4               1/1       Running   0          25m
default       azure-dl-6bcd9d5bfb-jjjfv               1/1       Running   0          25m
kube-system   azureproxy-79c5db744-j5hj4              1/1       Running   3          6d
kube-system   heapster-55f855b47-bs86x                2/2       Running   0          6d
kube-system   kube-dns-v20-7c556f89c5-96dfd           3/3       Running   0          6d
kube-system   kube-dns-v20-7c556f89c5-fm7v2           3/3       Running   0          6d
kube-system   kube-proxy-225hz                        1/1       Running   0          6d
kube-system   kube-proxy-g5x5p                        1/1       Running   0          6d
kube-system   kube-proxy-hw7fk                        1/1       Running   0          6d
kube-system   kube-proxy-ttnsh                        1/1       Running   0          6d
kube-system   kube-proxy-tvnp

If anything goes wrong you can use the commands below to observe the events on the node as well as review the logs.

In [29]:
!kubectl get events

LAST SEEN   FIRST SEEN   COUNT     NAME                                         KIND         SUBOBJECT                   TYPE      REASON                       SOURCE                              MESSAGE
25m         25m          1         azure-dl-6bcd9d5bfb-2txj4.1538679fef02be92   Pod                                      Normal    Scheduled                    default-scheduler                   Successfully assigned azure-dl-6bcd9d5bfb-2txj4 to aks-nodepool1-13347348-2
25m         25m          1         azure-dl-6bcd9d5bfb-2txj4.1538679ffc604553   Pod                                      Normal    SuccessfulMountVolume        kubelet, aks-nodepool1-13347348-2   MountVolume.SetUp succeeded for volume "default-token-lls54" 
22m         25m          2         azure-dl-6bcd9d5bfb-2txj4.153867a04b38abca   Pod          spec.containers{azure-dl}   Normal    Pulling                      kubelet, aks-nodepool1-13347348-2   pulling image "fboylu/kerasres50-cpu"
22m         22m          1      

26m         26m          1         azure-dl-7fdd7b85d5-nn2zm.15386793db6d13c7   Pod          spec.containers{azure-dl}   Normal    Killing                      kubelet, aks-nodepool1-13347348-2   Killing container with id docker://azure-dl:Need to kill Pod
26m         26m          1         azure-dl-7fdd7b85d5-nxcbl.15386793d8a6a7ce   Pod          spec.containers{azure-dl}   Normal    Killing                      kubelet, aks-nodepool1-13347348-1   Killing container with id docker://azure-dl:Need to kill Pod
26m         26m          1         azure-dl-7fdd7b85d5-pvkns.15386794224a3c13   Pod          spec.containers{azure-dl}   Normal    Killing                      kubelet, aks-nodepool1-13347348-0   Killing container with id docker://azure-dl:Need to kill Pod
26m         26m          1         azure-dl-7fdd7b85d5-r4d9z.15386793d4f6a7f1   Pod          spec.containers{azure-dl}   Normal    Killing                      kubelet, aks-nodepool1-13347348-2   Killing container with id docker:

Check the logs for the application pod.

In [30]:
pod_json = !kubectl get pods -o json
pod_dict = json.loads(''.join(pod_json))

In [31]:
!kubectl logs {pod_dict['items'][0]['metadata']['name']}

2018-06-15 18:30:42,485 CRIT Supervisor running as root (no user in config file)
2018-06-15 18:30:42,487 INFO supervisord started with pid 1
2018-06-15 18:30:43,489 INFO spawned: 'program_exit' with pid 9
2018-06-15 18:30:43,491 INFO spawned: 'nginx' with pid 10
2018-06-15 18:30:43,493 INFO spawned: 'gunicorn' with pid 11
2018-06-15 18:30:44,597 INFO success: program_exit entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
2018-06-15 18:30:45.344927: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2018-06-15 18:30:49,350 INFO success: nginx entered RUNNING state, process has stayed up for > than 5 seconds (startsecs)

102858752/102853048 [==============================] - 2s 0us/step
{"level": "INFO", "path": "/code/driver.py", "host": "azure-dl-6bcd9d5bfb-2txj4", "timestamp": "2018-06-15T18:30:56.674076Z", "tags": [], "message": "Model loading time: 11361.6 ms", 

In [32]:
!kubectl get deployment

NAME       DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
azure-dl   2         2         2            2           25m


It can take a few minutes for the service to populate the EXTERNAL-IP field below. This will be the IP you use to call the service. You can also specify an IP to use, please see the AKS documentation for further details.

In [33]:
!kubectl get service azure-dl

NAME       TYPE           CLUSTER-IP     EXTERNAL-IP      PORT(S)        AGE
azure-dl   LoadBalancer   10.0.210.143   168.62.183.212   80:31468/TCP   25m


Next, we will [test our web application deployed on AKS](05_TestWebApp.ipynb). Once, we are done with all the notebooks of the tutorial, below instructions can be used to delete the cluster and free resources.

# Scaling

In [51]:
!az aks scale --resource-group=$resource_group --name=$aks_name --node-count 3

{- Finished ..
  "agentPoolProfiles": [
    {
      "count": 3,
      "dnsPrefix": null,
      "fqdn": null,
      "name": "nodepool1",
      "osDiskSizeGb": null,
      "osType": "Linux",
      "ports": null,
      "storageProfile": "ManagedDisks",
      "vmSize": "Standard_NC6",
      "vnetSubnetId": null
    }
  ],
  "dnsPrefix": "fbAKSClust-fbaksrg-edf507",
  "fqdn": "fbaksclust-fbaksrg-edf507-e12d7f40.hcp.eastus.azmk8s.io",
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourcegroups/fbaksrg/providers/Microsoft.ContainerService/managedClusters/fbAKSClustergpu",
  "kubernetesVersion": "1.9.6",
  "linuxProfile": {
    "adminUsername": "azureuser",
    "ssh": {
      "publicKeys": [
        {
          "keyData": "ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQDTZYQFHNstYCR25qtvMrC6baTMS6TobaIRbgd0xOoafDy+2uBk0DMJuhGWoOcrsCnvadp5k/0K8qBRysyhlQGWb6+r8fBunThy+zpTKqdh3W8Q1y5UtKnGwwU1cqGXDOPUIXJYNPJqUKV829+MOrZjUynhHgSzDbY2ncGyoT+Farsvm01aGEdDapa+XRl4JAwtN1bb9q+Ii5y+MkpIOhLRMwATl05

In [34]:
!kubectl get nodes

NAME                       STATUS    ROLES     AGE       VERSION
aks-nodepool1-13347348-0   Ready     agent     6d        v1.9.6
aks-nodepool1-13347348-1   Ready     agent     6d        v1.9.6
aks-nodepool1-13347348-2   Ready     agent     6d        v1.9.6
aks-nodepool1-13347348-3   Ready     agent     6d        v1.9.6
aks-nodepool1-13347348-4   Ready     agent     6d        v1.9.6


In [42]:
!kubectl scale --current-replicas=20 --replicas=35 deployment/azure-dl

deployment.extensions "azure-dl" scaled


In [44]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
default       azure-dl-6bcd9d5bfb-2m8b4               1/1       Running   0          36m
default       azure-dl-6bcd9d5bfb-2pfph               1/1       Running   0          8m
default       azure-dl-6bcd9d5bfb-2txj4               1/1       Running   0          1h
default       azure-dl-6bcd9d5bfb-5hdpn               1/1       Running   0          25m
default       azure-dl-6bcd9d5bfb-69gz8               1/1       Running   0          36m
default       azure-dl-6bcd9d5bfb-6bdmj               1/1       Running   0          8m
default       azure-dl-6bcd9d5bfb-6qtjq               1/1       Running   0          36m
default       azure-dl-6bcd9d5bfb-75tw9               1/1       Running   0          8m
default       azure-dl-6bcd9d5bfb-7jjs5               1/1       Running   0          25m
default       azure-dl-6bcd9d5bfb-bx54r               1/1       Running   0          25m
default       azure-dl-6b

In [46]:
!kubectl get deployment

NAME       DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
azure-dl   35        35        35           35          1h


In [61]:
!az aks scale --resource-group=$resource_group --name=$aks_name --node-count 1

{- Finished ..
  "agentPoolProfiles": [
    {
      "count": 1,
      "dnsPrefix": null,
      "fqdn": null,
      "name": "nodepool1",
      "osDiskSizeGb": null,
      "osType": "Linux",
      "ports": null,
      "storageProfile": "ManagedDisks",
      "vmSize": "Standard_NC6",
      "vnetSubnetId": null
    }
  ],
  "dnsPrefix": "fbAKSClust-fbaksrg-edf507",
  "fqdn": "fbaksclust-fbaksrg-edf507-e12d7f40.hcp.eastus.azmk8s.io",
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourcegroups/fbaksrg/providers/Microsoft.ContainerService/managedClusters/fbAKSClustergpu",
  "kubernetesVersion": "1.9.6",
  "linuxProfile": {
    "adminUsername": "azureuser",
    "ssh": {
      "publicKeys": [
        {
          "keyData": "ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQDTZYQFHNstYCR25qtvMrC6baTMS6TobaIRbgd0xOoafDy+2uBk0DMJuhGWoOcrsCnvadp5k/0K8qBRysyhlQGWb6+r8fBunThy+zpTKqdh3W8Q1y5UtKnGwwU1cqGXDOPUIXJYNPJqUKV829+MOrZjUynhHgSzDbY2ncGyoT+Farsvm01aGEdDapa+XRl4JAwtN1bb9q+Ii5y+MkpIOhLRMwATl05

# Tear it all down

Once you are done with your cluster you can use the following two commands to destroy it all.

In [10]:
!kubectl delete -f az-dl.json

deployment.apps "azure-dl" deleted
service "azure-dl" deleted


In [3]:
!az aks delete -n $aks_name -g $resource_group -y

In [4]:
!az group delete --name $resource_group -y

 - Finished ..